# Wyckoff-Elliott Trading Bot - Backtest Example

This notebook demonstrates how to backtest the Wyckoff-Elliott signal strategy on historical data.

In [ ]:
# Import required libraries
import sys
import asyncio
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Add src to path
sys.path.append('../..')

from src.backtest.engine import BacktestEngine, print_backtest_results
from src.ingest.binance_rest import BinanceRESTClient

# Set plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Load Historical Data

Download historical candles from Binance public API.

In [ ]:
# Configuration
SYMBOL = 'BTCUSDT'
TIMEFRAME = '1h'
DAYS_BACK = 90

# Initialize Binance REST client
binance = BinanceRESTClient()

# Calculate date range
end_date = datetime.now()
start_date = end_date - timedelta(days=DAYS_BACK)

print(f"Loading {SYMBOL} {TIMEFRAME} data from {start_date.date()} to {end_date.date()}...")

# Download candles
candles = await binance.get_historical_klines(
    symbol=SYMBOL,
    interval=TIMEFRAME,
    start_time=start_date,
    end_time=end_date,
    limit=1500
)

print(f"Loaded {len(candles)} candles")

# Convert to DataFrame for visualization
df = pd.DataFrame(candles)
df['datetime'] = pd.to_datetime(df['open_time'], unit='ms')
df.set_index('datetime', inplace=True)

df.head()

## 2. Visualize Price Data

In [ ]:
# Plot OHLC data
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Price chart
ax1.plot(df.index, df['close'], label='Close Price', color='blue', linewidth=1.5)
ax1.fill_between(df.index, df['low'], df['high'], alpha=0.2, color='gray', label='High-Low Range')
ax1.set_ylabel('Price (USDT)', fontsize=12)
ax1.set_title(f'{SYMBOL} {TIMEFRAME} Price Chart', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Volume chart
ax2.bar(df.index, df['volume'], color='green', alpha=0.6, label='Volume')
ax2.set_ylabel('Volume', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Run Backtest

In [ ]:
# Initialize backtest engine
engine = BacktestEngine(
    initial_balance=10000.0,
    position_size_percent=0.02,  # 2% per trade
    commission=0.001  # 0.1% commission
)

# Run backtest
results = await engine.run_backtest(
    candles=candles,
    symbol=SYMBOL,
    timeframe=TIMEFRAME,
    enable_wyckoff=True,
    enable_elliott=True,
    min_confidence=0.65
)

# Print results
print_backtest_results(results)

## 4. Visualize Backtest Results

In [ ]:
# Create equity curve
trades_df = pd.DataFrame(results['trades'])

if len(trades_df) > 0:
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    # 1. Equity curve
    ax1 = axes[0]
    equity = [engine.initial_balance] + trades_df['balance_after'].tolist()
    ax1.plot(range(len(equity)), equity, color='blue', linewidth=2)
    ax1.axhline(y=engine.initial_balance, color='gray', linestyle='--', alpha=0.5, label='Initial Balance')
    ax1.set_ylabel('Account Balance ($)', fontsize=12)
    ax1.set_title('Equity Curve', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Drawdown
    ax2 = axes[1]
    peak = pd.Series(equity).expanding().max()
    drawdown = ((pd.Series(equity) - peak) / peak) * 100
    ax2.fill_between(range(len(drawdown)), drawdown, 0, color='red', alpha=0.4)
    ax2.set_ylabel('Drawdown (%)', fontsize=12)
    ax2.set_title('Drawdown Chart', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # 3. Trade PnL distribution
    ax3 = axes[2]
    ax3.bar(range(len(trades_df)), trades_df['pnl'], 
            color=['green' if pnl > 0 else 'red' for pnl in trades_df['pnl']],
            alpha=0.6)
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax3.set_xlabel('Trade Number', fontsize=12)
    ax3.set_ylabel('PnL ($)', fontsize=12)
    ax3.set_title('Individual Trade PnL', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No trades executed in backtest.")

## 5. Trade Analysis

In [ ]:
if len(trades_df) > 0:
    # Trade statistics by type
    print("\n=== Trade Statistics by Type ===")
    print(trades_df.groupby('type')['pnl'].agg(['count', 'mean', 'sum', 'min', 'max']))
    
    # Exit reasons
    print("\n=== Exit Reasons ===")
    print(trades_df['exit_reason'].value_counts())
    
    # Display first few trades
    print("\n=== Sample Trades ===")
    display(trades_df[['type', 'entry_price', 'exit_price', 'exit_reason', 'pnl', 'confidence']].head(10))
else:
    print("No trades to analyze.")

## 6. Parameter Optimization (Optional)

Test different confidence thresholds.

In [ ]:
# Test multiple confidence thresholds
confidence_levels = [0.55, 0.60, 0.65, 0.70, 0.75]
optimization_results = []

print("Running parameter optimization...\n")

for conf in confidence_levels:
    result = await engine.run_backtest(
        candles=candles,
        symbol=SYMBOL,
        timeframe=TIMEFRAME,
        enable_wyckoff=True,
        enable_elliott=True,
        min_confidence=conf
    )
    
    optimization_results.append({
        'confidence': conf,
        'total_trades': result['total_trades'],
        'win_rate': result['win_rate'],
        'total_pnl_percent': result['total_pnl_percent'],
        'max_drawdown': result['max_drawdown']
    })
    
    print(f"Confidence: {conf:.2f} | Trades: {result['total_trades']} | "
          f"Win Rate: {result['win_rate']:.2%} | PnL: {result['total_pnl_percent']:.2f}%")

# Plot optimization results
opt_df = pd.DataFrame(optimization_results)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].plot(opt_df['confidence'], opt_df['total_trades'], marker='o')
axes[0, 0].set_title('Total Trades vs Confidence')
axes[0, 0].set_xlabel('Confidence Threshold')
axes[0, 0].set_ylabel('Total Trades')

axes[0, 1].plot(opt_df['confidence'], opt_df['win_rate'] * 100, marker='o', color='green')
axes[0, 1].set_title('Win Rate vs Confidence')
axes[0, 1].set_xlabel('Confidence Threshold')
axes[0, 1].set_ylabel('Win Rate (%)')

axes[1, 0].plot(opt_df['confidence'], opt_df['total_pnl_percent'], marker='o', color='blue')
axes[1, 0].set_title('Total PnL vs Confidence')
axes[1, 0].set_xlabel('Confidence Threshold')
axes[1, 0].set_ylabel('Total PnL (%)')

axes[1, 1].plot(opt_df['confidence'], opt_df['max_drawdown'], marker='o', color='red')
axes[1, 1].set_title('Max Drawdown vs Confidence')
axes[1, 1].set_xlabel('Confidence Threshold')
axes[1, 1].set_ylabel('Max Drawdown (%)')

plt.tight_layout()
plt.show()